# Retrieving DAO tokens and pools data

We use **The Graph Explorer** to access the Uniswap subgraph and analyze the data related to a list of DAO tokens.

### Install dependencies

In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Define global variables

In [2]:
ENDPOINT = 'https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'
DAO_TOKENS = [
    'FWB',
    'BIT',
    'BANK',
    'OHM',
    'UNI',
    'CRV',
    'AAVE',
    'MKR',
    'DASH',
    'COMP',
    'SUSHI',
    'SNX',
    'ZRX'
]

CURRENCY_TOKENS = [
    'WETH',
    'USDC',
    'USDT',
    'DAI'
]

### Define private methods for querying

In [10]:
def _send_request(query, args=None):
    '''Make an API call to a given endpoint str, given a query str'''

    data = {'query': query, 'variables': args}
    
    try:
        r = requests.post(ENDPOINT, json=data)
    except requests.exceptions.HTTPError  as e:
        raise Exception('Error sending a request to {0}: {1}'.format(ENDPOINT, e.response.text))
    
    if r.status_code == 200:
        return r.json()
    else:
        raise Exception('Query failed - return code:{}.'.format(r.status_code))

In [11]:
 def _sanitize(response, key):
    '''Make sure the response dict can be parsed'''
    
    try:
        return response[key]
    except KeyError as e:
         raise Exception('Response cannot be parsed.'.format(e))
    

In [12]:
def _get_token_ids(token_list):
    '''Request a token id list'''
    
    query = """query($sym: String!) 
          {
          tokens(where: {symbol: $sym })
            {
            id
            }
          }
      """
        
    tokens = pd.DataFrame()
    for token in token_list:
        variables = {'sym': token}
        response = _send_request(query, variables)

        data = _sanitize(_sanitize(response, 'data'), 'tokens')
        
        id_list = []
        for token_id in data:
            id_list.append({'ID': _sanitize(token_id, 'id'), 'SYMBOL': token})

        tokens = tokens.append(id_list)
    
    return tokens

### Get DAO tokens' IDs

In [14]:
dao_token_list = _get_token_ids(DAO_TOKENS)
dao_token_list

,ID,SYMBOL
0,0x35bd01fc9d6d5d81ca9e055db88dc49aa2c699a8,FWB
0,0x1a4b46696b2bb4794eb3d4c26f1c55f9170fa4c5,BIT
0,0x24a6a37576377f63f194caa5f518a60f45b42921,BANK
1,0x2d94aa3e47d9d5024503ca8491fce9a2fb4da198,BANK
0,0x383518188c0c6d7730d91b2c03a03c837814a899,OHM
1,0x64aa3364f17a4d01c6f1751fd97c2bd3d7e7f1d5,OHM
0,0x1f9840a85d5af5bf1d1762f925bdaddc4201f984,UNI
1,0x43b94ba4e0a26d273b13214f6cb3e7c4668f8c4c,UNI
2,0xe6877ea9c28fbdec631ffbc087956d0023a76bf2,UNI
0,0xd533a949740bb3306d119cc777fa900ba034cd52,CRV


### Get currency tokens' IDs

In [17]:
currency_token_list = _get_token_ids(CURRENCY_TOKENS)
currency_token_list

,ID,SYMBOL
0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,WETH
0,0x566957ef80f9fd5526cd2bef8be67035c0b81130,USDC
1,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,USDC
0,0x75af3fd37d7ca74447e865f792f263b2b1159b58,USDT
1,0xdac17f958d2ee523a2206206994597c13d831ec7,USDT
0,0x6b175474e89094c44da98b954eedeac495271d0f,DAI
1,0x89d24a6b4ccb1b6faa2625fe562bdd9a23260359,DAI


### Get pool prices

In [19]:
def get_pool_price(dao_token_list, currency_token_list):
    ''''''
    
    query = """query($a: String!, $b: String!) 
      {
      pools(where: {token0: $a, token1: $b}){
          token0 {
              symbol
          }
          token1 {
              symbol
          }
          token1Price
          }
      }"""

    price_list = pd.DataFrame()
    
    for currency_token in currency_token_list['ID']:
        for dao_token in dao_token_list['ID']:
            
            variables = {"a": currency_token, "b": dao_token}
            response = _send_request(query, variables)
            response = _sanitize(_sanitize(response, 'data'), 'pools')
            
            if len(response) != 0:
                # todo: add for case len > 1
                price = _sanitize(response[0], 'token1Price')
                symbol = _sanitize(_sanitize(response[0], 'token1'), 'symbol')
                pair = _sanitize(_sanitize(response[0], 'token0'), 'symbol')
                price_list = price_list.append(
                    {
                        'PRICE': price, 
                        'SYMBOL': symbol,
                        'PAIR': pair
                    }, ignore_index=True)
    
    return price_list



In [20]:
get_pool_price(dao_token_list, currency_token_list)

,PAIR,PRICE,SYMBOL
0,WETH,5227.817929841012726677785900036214,UNI
1,WETH,994.3956107136813258392001674290013,CRV
2,WETH,4665.01958474423080842050569239396,ZRX
3,USDC,0.3150491977712207199170825635173666,CRV
4,USDC,0.005903832793910837426901448751020095,COMP
5,USDC,0.2691860049403619142285410723828019,SNX
6,USDC,1.707777463720752425584179344919104,ZRX
7,USDT,1.004480903358642128128897659540191,ZRX
8,DAI,0.0004988822717045862839645160485684384,MKR
9,DAI,0,COMP
